In [ ]:
from pyrealm.pmodel import (
    SubdailyScaler,
    memory_effect,
    SubdailyPModel,
    PModelEnvironment,
    PModel,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.pmodel.optimal_chi import OptimalChiPrentice14
from pyrealm.pmodel.functions import calc_ftemp_arrh, calc_ftemp_kphio,calc_modified_arrhenius_factor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.constants import PModelConst
pmodel_const = PModelConst()
from pyrealm.pmodel import (
    SubdailyScaler,
    memory_effect,
    SubdailyPModel,
    PModelEnvironment,
    PModel,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.pmodel.optimal_chi import OptimalChiPrentice14
from pyrealm.pmodel.functions import calc_ftemp_arrh, calc_ftemp_kphio, calc_modified_arrhenius_factor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.constants import PModelConst

pmodel_const = PModelConst()

# Get enzyme kinetics parameters
a, b, ha, hd = pmodel_const.kattge_knorr_kinetics
# Calculate entropy as a function of temperature _in °C_
deltaS = a + b * 10

# Number of half-hourly data points
n_pts = 17520  # 365 days * 48 half-hours

# Build a datetime index at 30-minute intervals
start_datetime = pd.to_datetime('2023-01-01')
datetimes = pd.date_range(start=start_datetime, periods=n_pts, freq='30T')

# Convert to a NumPy datetime64[m] array
datetimes_np = datetimes.to_numpy(dtype='datetime64[m]')

# Atmospheric pressure (Pa)
patm = np.full(n_pts, 101325)  # Standard atmospheric pressure

# VPD (vapor pressure deficit, Pa)
vpd = np.full(n_pts, 500)  # Assume VPD = 500 Pa

# fAPAR (unitless)
fapar = np.full(n_pts, 1)  # Assume full canopy cover

# PPFD (µmol m⁻² s⁻¹)
ppfd = np.full(n_pts, 1800)  # PPFD fixed

# Atmospheric CO₂ concentration (ppm)
co2 = np.full(n_pts, 400)  # One value per half hour

# List of acclimation temperatures
acclim_temps = np.arange(10, 41, 5)

# Create SubdailyScaler and set the acclimation window
fsscaler = SubdailyScaler(datetimes=datetimes_np)
fsscaler.set_window(
    window_center=np.timedelta64(12, 'h'),   # 12:00 noon
    half_width=np.timedelta64(30, 'm'),      # ±0.5 hour
)

# Create a mask for 11:30–12:30 (acclimation window)
times = datetimes.time
start_time = pd.to_datetime('11:30').time()
end_time = pd.to_datetime('12:30').time()
acclim_mask = (times >= start_time) & (times <= end_time)

# Initialize containers for results
gpp_results = []
vcmax_results = []
jmax_results = []
temp_results = []
ac_results = []
aj_results = []
results = []

# Loop over acclimation temperatures
for acclim_temp in acclim_temps:
    print(f"\nProcessing acclimation temperature: {acclim_temp}°C")
    
    # Air temperature (°C) from 0°C to 50°C, n_pts points
    tc_1d = np.linspace(0, 50, n_pts)

    # Acclimated values (daily means)
    co2_acclim = fsscaler.get_daily_means(co2)
    vpd_acclim = fsscaler.get_daily_means(vpd)
    patm_acclim = fsscaler.get_daily_means(patm)
    fapar_acclim = fsscaler.get_daily_means(fapar)

    # Set temperature to the current acclimation temperature in the window
    tc_1d[acclim_mask] = acclim_temp
    ppfd[acclim_mask] = 1800
    
    # Create PModelEnvironment
    pmodel_env = PModelEnvironment(
        tc=tc_1d,
        patm=patm,
        vpd=vpd,
        co2=co2,
    )

    # Subdaily C3 model
    subdailyC3 = SubdailyPModel(
        env=pmodel_env,
        method_kphio='temperature',
        reference_kphio=0.125,
        method_optchi='prentice14',
        fapar=fapar,
        ppfd=ppfd,
        fs_scaler=fsscaler,
        alpha=1 / 15,
        allow_holdover=True,
    )
    
    # Extract GPP, Vcmax, and Jmax
    # Constants for potential temperature-response use

    vcmax = subdailyC3.subdaily_vcmax
    jmax = subdailyC3.subdaily_jmax
    gpp = subdailyC3.gpp/12     # convert to umol
    ac = subdailyC3.subdaily_Ac
    aj = subdailyC3.subdaily_Aj

    mask = ~np.isnan(gpp) & ~np.isnan(vcmax) & ~np.isnan(jmax)
    tc_1d_clean = tc_1d[mask]
    gpp_clean = gpp[mask]
    vcmax_clean = vcmax[mask]
    jmax_clean = jmax[mask]
    ac_clean = ac[mask]
    aj_clean = aj[mask]

    # Drop points where instantaneous T equals the acclimation T (window times)
    mask_tc = tc_1d_clean != acclim_temp
    tc_1d_clean = tc_1d_clean[mask_tc]
    gpp_clean = gpp_clean[mask_tc]
    vcmax_clean = vcmax_clean[mask_tc]
    jmax_clean = jmax_clean[mask_tc]
    ac_clean = ac_clean[mask_tc]
    aj_clean = aj_clean[mask_tc]

    # Store outputs
    temp_results.append(tc_1d_clean)
    gpp_results.append(gpp_clean)
    vcmax_results.append(vcmax_clean)
    jmax_results.append(jmax_clean)
    ac_results.append(ac_clean)   # <— previously appended vcmax_clean by mistake
    aj_results.append(aj_clean)   # <— previously appended jmax_clean by mistake

    results.append({
        'acclim_temp': acclim_temp,
        'instant_temps': tc_1d_clean.copy(),
        'gpp_values': np.array(gpp_clean),
        'a':np.array(gpp_clean),
        'jmax':np.array(jmax_clean),
        'vcmax':np.array(vcmax_clean),
        'ac':np.array(ac_clean),
        'aj':np.array(aj_clean)
    })


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
# ---------- Configuration ----------
light_levels = [600, 1200, 1800]      # acclimation PPFD levels
temps = list(acclim_temps)            # acclimation temperature array, e.g., np.arange(10,40,1)

# Colors: continuous colormap for temperature (cold → hot)
norm = mcolors.Normalize(vmin=min(temps), vmax=max(temps))
cmap = cm.get_cmap('coolwarm')

# Store results for different light scenarios
results_by_light = {L: [] for L in light_levels}

# ---------- Main loop (outer: light; inner: temperature) ----------
for acclim_ppfd in light_levels:
    for acclim_temp in temps:
        # 1) Construct instantaneous forcing — PPFD always 1800, not depending on acclim_ppfd
        tc_1d = np.linspace(-10, 60, n_pts).copy()
        patm_use = patm.copy()
        vpd_use  = vpd.copy()
        co2_use  = co2.copy()
        fapar_use = fapar.copy()
        ppfd_use  = np.full(n_pts, 1800.0)    
        tc_1d[acclim_mask] = acclim_temp      
        ppfd_use[acclim_mask] = acclim_ppfd    

        # Daily means (used for acclimation)
        co2_acclim  = fsscaler.get_daily_means(co2_use)
        vpd_acclim  = fsscaler.get_daily_means(vpd_use)
        patm_acclim = fsscaler.get_daily_means(patm_use)

        # Environment and model: instantaneous PPFD fixed at 1800
        env = PModelEnvironment(tc=tc_1d, patm=patm_use, vpd=vpd_use, co2=co2_use)
        subdailyC3 = SubdailyPModel(
            env=env,
            method_kphio='temperature',
            reference_kphio=0.125,
            method_optchi='prentice14',
            fapar=fapar_use,
            ppfd=ppfd_use, 
            fs_scaler=fsscaler,
            alpha=1/15,
            allow_holdover=True,
        )

        # 2) Acclimation state uses acclim_ppfd (to reflect "acclimated light level")
        acclim_env = PModelEnvironment(
            tc=acclim_temp,
            patm=patm_acclim,
            vpd=vpd_acclim,
            co2=co2_acclim
        )
        pmodel_acclim = PModel(
            env=acclim_env,
            method_kphio='temperature',
            reference_kphio=0.125,
            method_optchi='prentice14'
        )
        pmodel_acclim.estimate_productivity(fapar=1, ppfd=acclim_ppfd)  # <<< use acclim_ppfd only in acclimation

        # 3) Extract outputs and clean
        gpp = subdailyC3.gpp / 12.0
        mask = np.isfinite(gpp)
        instant_temps = tc_1d[mask]
        gpp_clean = gpp[mask]
        # Remove the acclimation-window temperature points
        sel = instant_temps != acclim_temp
        instant_temps = instant_temps[sel]
        gpp_clean = gpp_clean[sel]

        results_by_light[acclim_ppfd].append({
            'acclim_temp': acclim_temp,
            'instant_temps': instant_temps,
            'gpp_values': gpp_clean
        })


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
####### figure a
# ---------- Configuration ----------
light_levels = [1800]  # Only keep 1800
all_temps = sorted({rec['acclim_temp'] for rec in results_by_light.get(1800, [])})

# Define 9 fixed growth temperature points (based on acclim_temps)
all_temps = np.linspace(min(acclim_temps), max(acclim_temps), 9, dtype=int)
palette_temps = np.linspace(min(acclim_temps), max(acclim_temps), 9, dtype=int)

# Color palette for different growth temperatures
colors9 = [
    "#0b0c48",  # very dark blue
    "#313695",  # dark blue
    "#4575b4",  # blue
    "#74add1",  # light blue
    "#abd9e9",  # pale blue
    "#F9D057",  # golden yellow
    "#fdae61",  # orange
    "#f46d43",  # reddish orange
    "#a50026"   # dark red
]
temp_palette = dict(zip(palette_temps, colors9))

def color_for_temp(t):
    """Find the nearest temperature in the palette and return its color."""
    t = int(round(t))
    nearest = palette_temps[np.argmin(np.abs(palette_temps - t))]
    return temp_palette[int(nearest)]

# ---------- Plotting ----------
fig, ax = plt.subplots(figsize=(6, 5))

# Plot curves for growth light = 1800
curves = results_by_light.get(1800, [])
for rec in sorted(curves, key=lambda d: d['acclim_temp']):
    t = rec['acclim_temp']
    x = rec['instant_temps']
    y = rec['gpp_values']
    ax.plot(x, y, color=color_for_temp(t), linewidth=2, label=f"{int(round(t))}°C")

# Style adjustments
ax.tick_params(axis='both', direction='in', length=6, width=1.2, labelsize=16)
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# Axis labels and limits
ax.set_xlim(0, 50)
ax.set_ylim(0, 120)
ax.set_ylabel('GPP (µmol m$^{-2}$ s$^{-1}$)', fontsize=18, labelpad=8)
ax.set_xlabel('Instantaneous Temperature (°C)', fontsize=18, labelpad=8)

# Text label in the top-left corner
ax.text(0.05, 0.98, 'Growth Light 1800', transform=ax.transAxes,
        ha='left', va='top', fontsize=15, color='black')

# Legend on the right
handles, labels = ax.get_legend_handles_labels()
pairs = sorted({lbl: h for h, lbl in zip(handles, labels)}.items(),
               key=lambda z: int(z[0].rstrip("°C")))
fig.legend([p[1] for p in pairs], [p[0] for p in pairs],
           title="Growth Temperature", fontsize=14, title_fontsize=15,
           frameon=False, loc="center right", bbox_to_anchor=(1.15, 0.5))

# Add subplot label
fig.text(0.01, 0.99, "(a)", ha="left", va="top", fontsize=20)

# Adjust layout to leave space for legend
fig.tight_layout(rect=[0, 0, 0.9, 1])
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
####### figure b

# ---- Fixed 9-color palette (deep blue → red, non-fluorescent) ----
colors9 = [
    "#0b0c48",  # very dark blue
    "#313695",  # dark blue
    "#4575b4",  # blue
    "#74add1",  # light blue
    "#abd9e9",  # pale blue
    "#F9D057",  # golden yellow
    "#fdae61",  # orange
    "#f46d43",  # red-orange
    "#a50026"   # dark red
]

# Map (up to) 9 representative growth temperatures to colors
temps_all = sorted({r['acclim_temp'] for r in results})
if len(temps_all) >= 9:
    key_temps = np.linspace(min(temps_all), max(temps_all), 9, dtype=int)
else:
    key_temps = np.array(temps_all, dtype=int)

temp_colors = {int(t): colors9[i] for i, t in enumerate(key_temps[:len(colors9)])}

def color_for_temp(t):
    """Return color for a growth temperature by snapping to nearest key_temps value."""
    t_int = int(round(t))
    nearest = key_temps[np.argmin(np.abs(key_temps - t_int))]
    return temp_colors[int(nearest)]

# ---- Find Ac==Aj switch temperatures (linear interpolation) ----
def find_switch_points(T, Ac, Aj):
    diff = Ac - Aj
    sgn = np.sign(diff)
    idx = np.where(np.diff(sgn) != 0)[0]
    Ts = []
    for i in idx:
        x1, x2 = T[i], T[i+1]
        y1, y2 = diff[i], diff[i+1]
        if np.isfinite(y1) and np.isfinite(y2) and (y2 - y1) != 0:
            Tswitch = x1 - y1 * (x2 - x1) / (y2 - y1)
            Ts.append(Tswitch)
    return Ts

# ---- Three-panel plot ----
panel_titles = ["Rubisco-limited", "J-limited", "Assimilation (min)"]
panel_keys   = ["ac", "aj", "a"]
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5), sharex=True, sharey=True)

for ax, key, title in zip(axes, panel_keys, panel_titles):
    for rec in results:
        tacc = rec["acclim_temp"]

        # Extract series and mask finite values
        T  = np.asarray(rec["instant_temps"])
        Y  = np.asarray(rec[key])
        m  = np.isfinite(T) & np.isfinite(Y)
        if m.sum() == 0:
            continue
        T, Y = T[m], Y[m]

        col  = color_for_temp(tacc)

        # Common line settings
        lw = 2.4 if key == "a" else 1.5
        ls = "-"  if key == "a" else ("-" if key == "ac" else "--")

        # Distinct markers for switch points
        markers = ["o", "s", "D", "^", "v", "P", "X", "*"]

        if key == "a":
            # Compute Ac, Aj, Amin consistently with the same mask
            Ac_full = np.asarray(rec["ac"])
            Aj_full = np.asarray(rec["aj"])
            Ac = Ac_full[m]
            Aj = Aj_full[m]
            Amin = np.minimum(Ac, Aj)

            # Topt/Aopt (peak of Amin)
            imax = int(np.argmax(Amin))
            Topt = float(T[imax])
            Aopt = float(Amin[imax])

            # Draw Topt marker
            ax.scatter([Topt], [Aopt],
                       s=100, marker="*", facecolor="red", edgecolor="black",
                       linewidth=1.0, zorder=6)

            # Find switch points and segment lines by limitation
            Ts = sorted(find_switch_points(T, Ac, Aj))
            split_T = [T[0]] + Ts + [T[-1]]

            for i in range(len(split_T) - 1):
                mask_seg = (T >= split_T[i]) & (T <= split_T[i+1])
                if mask_seg.any():
                    # If Ac - Aj <= 0 on average → Ac limitation; else Aj limitation
                    ls_seg = "-" if np.nanmean(Ac[mask_seg] - Aj[mask_seg]) <= 0 else "--"
                    ax.plot(T[mask_seg], Amin[mask_seg], color=col, linewidth=2.4, linestyle=ls_seg)

            # Mark switch points with small black markers
            for k, Tswitch in enumerate(Ts):
                j = np.searchsorted(T, Tswitch)
                j = max(1, min(j, len(T)-1))
                x1, x2 = T[j-1], T[j]
                y1, y2 = Amin[j-1], Amin[j]
                # Linear interpolation for Amin at Tswitch
                Aint = y1 + (y2 - y1) * (Tswitch - x1) / (x2 - x1)
                ax.scatter([Tswitch], [Aint],
                           s=40, facecolor="black",
                           linewidth=1.2, zorder=5, marker=markers[k % len(markers)])
        else:
            # Plain lines for Ac and Aj panels
            ax.plot(T, Y, color=col, linewidth=1.5, linestyle=ls,
                    label=f"{int(round(tacc))}°C")

    # Corner labels (Ac/Aj/A)
    if key == "ac":
        ax.text(0.1, 0.98, "A$_c$", transform=ax.transAxes,
                ha="left", va="top", fontsize=18)
    elif key == "aj":
        ax.text(0.1, 0.98, "A$_j$", transform=ax.transAxes,
                ha="left", va="top", fontsize=18)
    else:
        ax.text(0.1, 0.98, "A = min(A$_c$, A$_j$)", transform=ax.transAxes,
                ha="left", va="top", fontsize=15)

    # PPFD note (only if not 1800)
    if rec.get("ppfd", 1800) != 1800:
        ax.text(0.9, 0.04, f"PPFD = {rec['ppfd']}", transform=ax.transAxes,
                ha="right", va="bottom", fontsize=12, color="black")

    # Axes styling
    ax.set_xlim(0, 50)
    ax.set_ylim(0, 120)
    ax.tick_params(axis='both', direction='in', length=6, width=1.2, labelsize=16)
    for sp in ["top", "right"]:
        ax.spines[sp].set_visible(False)

# Axis labels and figure label
axes[0].set_ylabel('Assimilation (µmol m$^{-2}$ s$^{-1}$)', fontsize=18, labelpad=8)
axes[1].set_xlabel('Instantaneous Temperature (°C)', fontsize=18, labelpad=8)
fig.text(0.01, 1.05, "(b)", ha="left", va="top", fontsize=20)

fig.tight_layout(rect=[0, 0, 0.9, 1])
plt.show()
